In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle

In [2]:
DATA_DIR = "dataset"
categories = ['Apple', 'Dianthus', 'Figs', 'Hibiscus', 'Rose']

data = []


def make_data():
    for category in categories:
        path = os.path.join(DATA_DIR, category)
        label = categories.index(category)

        for imgName in os.listdir(path):
            imagePath = os.path.join(path, imgName)
            image = cv2.imread(imagePath)

            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224, 224))

                image = np.array(image, dtype=np.float32)

                data.append([image, label])
            except Exception as e:
                pass
    #         cv2.imshow('Image :: ',image)
    #         break
    #     break
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    print(len(data))
    pik = open('data.pickle', 'wb')
    pickle.dump(data, pik)
    pik.close()


def load_data():
    pick = open('data.pickle', 'rb')
    data = pickle.load(pick)
    pick.close()

    np.random.shuffle(data)

    feature = []
    labels = []

    for img, label in data:
        feature.append(img)
        labels.append(label)

    feature = np.array(feature, dtype=np.float32)
    labels = np.array(labels)

    feature = feature/255.0

    return [feature, labels]


make_data()


662


In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
(feature, labels) = load_data()

# print(load_data())

x_train, x_test, y_train, y_test = train_test_split(
    feature, labels, test_size=0.1)


In [5]:
input_layer = tf.keras.layers.Input([224, 224, 3])

conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(
    5, 5), padding='Same', activation='relu')(input_layer)

pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(
    3, 3), padding='Same', activation='relu')(pool1)

pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv2)

conv3 = tf.keras.layers.Conv2D(filters=96, kernel_size=(
    3, 3), padding='Same', activation='relu')(pool2)

pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv3)

conv4 = tf.keras.layers.Conv2D(filters=96, kernel_size=(
    3, 3), padding='Same', activation='relu')(pool3)

pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv4)

flt1 = tf.keras.layers.Flatten()(pool4)

dnl = tf.keras.layers.Dense(512, activation='relu')(flt1)
out = tf.keras.layers.Dense(5, activation='softmax')(dnl)


In [6]:

model = tf.keras.Model(input_layer, out)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train,y_train,batch_size = 10, epochs = 5)
model.save('mymodel.h5')


Epoch 1/5
60/60 [==============================] - 22s 354ms/step - loss: 1.3496 - accuracy: 0.4437
Epoch 2/5
60/60 [==============================] - 21s 350ms/step - loss: 1.1328 - accuracy: 0.5345
Epoch 3/5
60/60 [==============================] - 21s 351ms/step - loss: 0.9855 - accuracy: 0.6084
Epoch 4/5
60/60 [==============================] - 21s 346ms/step - loss: 0.7819 - accuracy: 0.6992
Epoch 5/5
60/60 [==============================] - 22s 363ms/step - loss: 0.6173 - accuracy: 0.7580


In [7]:
from skimage import transform
import numpy as np
from PIL import Image
model = tf.keras.models.load_model('mymodel.h5')
# model.evaluate(x_test, y_test, verbose=1)
# from tensorflow.keras.utils import to_categorical


def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/224
    np_image = transform.resize(np_image, (224, 224, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image


imageForPrediction = "C:\\Users\\Jash\\Downloads\\pexels-pixabay-56866.jpg"
image = load(imageForPrediction)
# y_proba = model.predict(image)
# y_classes = tf.keras.np_utils.probas_to_classes(y_proba)

predictions = model.predict(image)
print(predictions)

maxAcc = float(-(sys.maxsize - 1))
pred = 'word'
i = 0
for label in categories:
    if(predictions[0][i] > maxAcc):
        pred = label
        maxAcc = predictions[0][i]
    i = i + 1
print("\t%s ==> %.2f Accuracy" % (label, 100*maxAcc))


# prediction = model.predict(x_test)

# plt.figure(figsize=(9,9))

# for i in range(9):
#     plt.subplot(3,3,i+1)
#     plt.imshow(x_test[i])
#     plt.xlabel('Actual: '+categories[y_test[i]]+'\n'+'Predicted: '+categories[np.argmax(prediction[i])])

#     plt.xticks([])
# plt.show()


1/1 [==============================] - 2s 2s/step
[[1.7110984e-12 7.7514822e-05 1.6260873e-16 9.2666005e-07 9.9992156e-01]]
	Rose ==> 99.99 Accuracy
